# Genereer buien

Notebook voor het aanmaken van BUI-bestanden voor RR-modellen

In [1]:
from delft3dfmpy import DFlowRRModel, DFlowRRWriter
import pandas as pd
import geopandas as gpd
import hydrotools
from pathlib import Path

data_path = Path(r"..\data").absolute().resolve()
hydrologische_eenheden = data_path.joinpath("hydrologische_eenheden",
                                            "HydrologischeEenheden_selectie.shp")
buien_dir = Path(r"..\modellen\buien\rr")

start_datetime = pd.Timestamp('2020-06-01 00:00:00')
end_datetime = start_datetime + pd.Timedelta(days=15)

drrmodel = DFlowRRModel()

precipitation = hydrotools.generate_meteo_series(7, start_datetime, end_datetime)
evaporation = hydrotools.generate_meteo_series(0, start_datetime, end_datetime)
meteo_stations = gpd.read_file(hydrologische_eenheden)["HEIDENT"].to_list()

for station in meteo_stations:
    drrmodel.external_forcings.add_precip(station, precipitation)
    drrmodel.external_forcings.add_evap(station, evaporation)   

rr_writer = DFlowRRWriter(drrmodel,
                          output_dir=r"modellen\buien",
                          name="buien")    
buien_dir.mkdir(exist_ok=True, parents=True)
rr_writer.write_meteo()